In [1]:
import numpy as np
import pandas as pd 
from scipy.special import expit 
from sklearn.linear_model import LogisticRegression
from keras.layers import Dense, Input
from keras.models import Model


Using TensorFlow backend.


In [3]:
N = 100000
inv_logit = expit
x1 = np.random.binomial(1, p=0.5, size=N) #Generating sunny/or not sunny days
x2 = np.random.binomial(1, p=inv_logit(-3.*x1)) #Sprinkler on/off depend on wether it's sunny or not
x3 = np.random.binomial(1, p=inv_logit(3.*x1)) #Rainy yes/no depend on wether it's sunny or not
x4 = np.bitwise_or(x2, x3) #Sidewalk is wet if it's raining or if sprinklers are on
x5 = np.random.binomial(1, p=inv_logit(3.*x4)) #Slippery or not depends on wether or not sidewalk is wet
X = pd.DataFrame({'$x_1$': x1, '$x_2$': x2, '$x_3$': x3, '$x_4$': x4, '$x_5$': x5})

The causal graph of the data generated above is the following:\n
<img src="./images/causal_graph.png">
Correlation matrix of variables in the system. Notice the negative relationship between x1 and x2, as well as between x2 and x3. This indicate that when the weather is good, the sprinkler is turned on and when it rains the sprinkler is turned off. 

In [11]:
X.corr()

,$x_1$,$x_2$,$x_3$,$x_4$,$x_5$
$x_1$,1.000000,-0.504735,0.507010,0.290376,0.144924
$x_2$,-0.504735,1.000000,-0.253646,0.255319,0.123463
$x_3$,0.507010,-0.253646,1.000000,0.679031,0.341176
$x_4$,0.290376,0.255319,0.679031,1.000000,0.499196
$x_5$,0.144924,0.123463,0.341176,0.499196,1.000000


The following shows the naive conditional expectation values for wether the sidewalk is wet given that the sprinkler is on. 

In [14]:
X.groupby('$x_2$').mean()[['$x_5$']]

,$x_5$
$x_2$,
0,0.858605
1,0.951524


We can express the probability distribution of $X_5$ conditional on the observed value of $X_2$ as:
$$
\begin{equation*}
P(X_1, X_2, X_3, X_4, X_5|X_2) = P(X_5|X_4)P(X_4|X_2,X_3)P(X_3|X_1)P(X_2|X_1)P(X_1)
\end{equation*}
$$
We wish to know the effect that sprinkler (on/off) has on how slippery the sidewalk is. This is the difference in the expected value of $X_5$ given the covariate $X_2$:
$$
\begin{equation*}
\sigma_{naive} = E[X_5 | X_2 = 1] -  E[X_5 | X_2 = 0]\\
\sigma_{naive} = 0.95 - 0.86\\
\sigma_{naive} = 0.09
\end{equation*}
$$
The sidewalk is 9 percentage points more likely to be slippery given that the sprinkler was on.

However this result does not show the true effect of $X_2$ on $X_5$, because of $X_2$'s dependence on $X_1$ in this dataset. So we create the following dataset:

In [7]:
x1 = np.random.binomial(1, p=0.5, size=N)

x2_0 = np.random.binomial(1, p=0, size=N)
x2_1 = np.random.binomial(1, p=1, size=N)

x3 = np.random.binomial(1, p=inv_logit(3.*x1))

x4_0 = np.bitwise_or(x2_0, x3)
x4_1 = np.bitwise_or(x2_1, x3)

x5_0 = np.random.binomial(1, p=inv_logit(3.*x4_0))
x5_1 = np.random.binomial(1, p=inv_logit(3.*x4_1))
 
Xn = pd.DataFrame({'$x2_0$': x2_0, '$x2_1$': x2_1, '$x5_0$': x5_0, '$x5_1$': x5_1})
Xn['$x5_1$'].mean() - Xn['$x5_0$'].mean()

0.12490000000000001

The dataset above is interesting because it elicits the power and usefulness of interventions. In this dataset the value of $X_2$ is independent from $X_1$. The causal link between $X_1$ and $X_2$ is broken, therefore the probability distributions on descendent variables of $X_1$ have also changed, as they no longer depend on $X_1$ (These descendent variables include $X_2$, $X_4$ and $X_5$). This dataset in effect depicts a different system from the first one where we simply observed $X_2$. Here the intervention alters the dependecies in the graph. The joint distribution after this intervention is the following:
$$
\begin{equation*}
P_{do(X_2 = x_2)}(X_1, X_2, X_3, X_4, X_5|X_2) = P(X_5|X_4)P(X_4|X_2=x_2,X_3)P(X_3|X_1)P(X_1)
\end{equation*}
$$
The expression above is an application of the more general Robins G-Formula:
$$
\begin{equation*}
P(X_j|do(X_i = x_i)) = \sum_Z P(X_j|X_i, Z)P(Z)
\end{equation*}
$$

In the result above we now get that the sidewalk in 12 percentage points more likely to be slippery given that the sprinkler was on. Let's now attempt to predict the effect of $X_2$ (sprinkler on/off) on $X_5$ (sidewalk slippery/or not).
We can use a regression model such that:
$$
\begin{equation}
X_5 = \beta_0 + \beta_1X_2
\end{equation}
$$
In this model we are doing the regression of $X_5$: (sidewalk slippery/or not), against the covariate $X_2$: (sprinkler on/off).

In [16]:
# build our model, predicting $x_5$ using $x_2$
model = LogisticRegression()
model = model.fit(X[['$x_2$']], X['$x_5$'])

# what would have happened if $x_2$ was always 0:
X0 = X.copy()
X0['$x_2$'] = 0
y_pred_0 = model.predict_proba(X0[['$x_2$']])

# what would have happened if $x_2$ was always 1:
X1 = X.copy()
X1['$x_2$'] = 1
y_pred_1 = model.predict_proba(X1[['$x_2$']])

# now, let's check the difference in probabilities
y_pred_1[:, 1].mean() - y_pred_0[:,1].mean()



0.09288453364376004

Result above is the same 0.09 difference as $\sigma_{naive}$. We wish to build a model that predicts an effect that is closer to the true effect. In order to this, we must take into consieration that $X_2$ depends on $X_1$: (Sunny dat/not sunny day) for its value. $X_1$ is therefore a cause of $X_2$. By performing a regression on both covariates we can account for more on the variation in $X_5$. The new regression model therefore is the following:
$$
\begin{equation}
X_5 = \beta_0 + \beta_1X_2 + \beta_2X_1
\end{equation}
$$


In [17]:
model = LogisticRegression()
model = model.fit(X[['$x_2$', '$x_1$']], X['$x_5$'])
 
# what would have happened if $x_2$ was always 0:
X0 = X.copy()
X0['$x_2$'] = 0
y_pred_0 = model.predict_proba(X0[['$x_2$', '$x_1$']])
 
# what would have happened if $x_2$ was always 1:
X1 = X.copy()
X1['$x_2$'] = 1
 
# now, let's check the difference in probabilities
y_pred_1 = model.predict_proba(X1[['$x_2$', '$x_1$']])
y_pred_1[:, 1].mean() - y_pred_0[:,1].mean()

0.14194231684053804

Regression on both $X_1$ and $X_2$ gives an estimate difference that is closer to the true effect of $X_2$. But our regression over-estimated the effect. Let's see if we can do better with a model that is more general than linear regression. We will use a deep feedforward architecture.

In [4]:
dense_size = 128
input_features = 2
 
x_in = Input(shape=(input_features,))
h1 = Dense(dense_size, activation='relu')(x_in)
h2 = Dense(dense_size, activation='relu')(h1)
h3 = Dense(dense_size, activation='relu')(h2)
y_out = Dense(1, activation='sigmoid')(h3)
 
model = Model(input=x_in, output=y_out)
model.compile(loss='binary_crossentropy', optimizer='adam') 
model.fit(X[['$x_1$', '$x_2$']].values, X['$x_5$'])

Epoch 1/1
100000/100000 [==============================] - 4s 44us/step - loss: 0.3237


Let's now perform the same prediction procedure as before

In [5]:
X_zero = X.copy()
X_zero['$x_2$'] = 0
x5_pred_0 = model.predict(X_zero[['$x_1$', '$x_2$']].values)
 
X_one = X.copy()
X_one['$x_2$'] = 1
x5_pred_1 = model.predict(X_one[['$x_1$', '$x_2$']].values)
 
x5_pred_1.mean() - x5_pred_0.mean()

0.11946136

We see that this result is different from the one given by the logistic regression model. We can now compute how much less likely is the sidewalk to be slippery if we turn off the sprinkler.

In [6]:
X['$x_5$'].mean() - x5_pred_0.mean()

0.06620157037734986

It will be 7 percent less likely that the sidewalk is slippery if a policy is passed to keep sprinklers off.